<a href="https://colab.research.google.com/github/thotran2015/COVID19_Causality/blob/master/covariate_adjustment_network_effect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
# !pip install -U plotly
# import plotly.express as px
from urllib.request import urlopen

import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import pandas
import time
import io
import json

#You will need to mount the drive to use the relevant datasets in the shared 'Data' folder
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
def load_data(ver=2, t_type='unacast'):
  if ver == 2:
    df_X = pd.read_csv('/content/drive/My Drive/COVID19 Data Exploration /Data/all_X_ver2.csv')
    del df_X['Unnamed: 0']
    df_Y = pd.read_csv('/content/drive/My Drive/COVID19 Data Exploration /Data/case_diff_Y_v2.csv')
    del df_Y['Unnamed: 0']
  elif ver == 1:
    df_X = pd.read_csv('/content/drive/My Drive/COVID19 Data Exploration /Data/all_X.csv')
    del df_X['Unnamed: 0']
    df_Y = pd.read_csv('/content/drive/My Drive/COVID19 Data Exploration /Data/case_diff_Y.csv')
    del df_Y['Unnamed: 0']

  if t_type == 'unacast':
    df_T = pd.read_csv('/content/drive/My Drive/COVID19 Data Exploration /Data/unacast_T.csv')
    del df_T['Unnamed: 0']
  elif t_type == 'safegraph':
    df_T = pd.read_csv('/content/drive/My Drive/COVID19 Data Exploration /Data/safegraph_data.csv')
  df_T = df_T.rename(columns = {'FIPS-Code':'fips', 'Social_Distance_Change':'score'})

  df_X['fips'] = df_X['fips'].astype(float).astype(int)
  df_Y['fips'] = df_Y['fips'].astype(float).astype(int)
  df_T['fips'] = df_T['fips'].astype(float).astype(int)

  return df_X, df_Y, df_T

In [0]:
import math
def construct_dataset(df_X, df_Y, df_T, Y_mode='adj_increase', population_adjustment_factor=1000., num_task=3, t_type='safegraph', \
                      average_y=True):
  """
  Returns dataset as a dict {}
  Key: FIPS
  Value: Tuple (X, Y, T) where X, Y, T are arrays
  """
  data = {}

  score = 'score' if t_type == 'safegraph' else 'unacast_score'
  for idx, row in df_X.iterrows():
    if len(df_Y.loc[df_Y['fips']==row['fips']]) == 0 or len(df_T.loc[df_T['fips']==row['fips']]) == 0 :
      continue
    if math.isnan(df_T.loc[df_T['fips']==row['fips']][score].values[0]):
      continue
    id = row['fips']
    x = row.values[3:]
    relevant_x = []  
    # relevant_x.append(x[0])         # ratio of male
    # relevant_x.append(sum(x[2:13])) # ratio of people under 60
    relevant_x.append(x[21]) # ratio (%) of african american population
    relevant_x.append(x[25]) # ratio of hispanic population
    # for k in x[20:26]:
    #  relevant_x.append(k) # race distribution 
    for i, k in enumerate(x[30:37]):
      if i==3 :
        continue
      relevant_x.append(k) # 31: poverty, 32: unemployment, 33: median income, 34: pop density, 35: total test per capita, 36: positive test rate, 37: democrat support rate
    relevant_x.append(float(x[39]) / float(x[26])) # current confirmed case / capita
    relevant_x.append(float(x[40]) / float(x[26])) # current confirmed death / capita

    if Y_mode == 'adj_increase':
      relevant_x.append(x[42]) # per capita increase during March 23-28
    else:
      relevant_x.append(x[41])

    y1_key = 'Y1_' + Y_mode
    y1 = df_Y.loc[df_Y['fips']==id][y1_key].values[0] / 7.0

    y2_key = 'Y2_' + Y_mode
    y2 = df_Y.loc[df_Y['fips']==id][y2_key].values[0] / 7.0

    y3_key = 'Y3_' + Y_mode
    y3 = df_Y.loc[df_Y['fips']==id][y3_key].values[0] / 7.0

    # average daily case within 5, 10, 15 days
    if average_y and Y_mode == 'adj_increase':
      y1 = y1
      y2 = (y1+y2) / 2.0
      y3 = (y3 + 2.0*y2) / 3.0

    if Y_mode == 'adj_increase':
      y1 *= population_adjustment_factor
      y2 *= population_adjustment_factor
      y3 *= population_adjustment_factor

    y = [y1, y2, y3] if num_task == 3 else ([y1, y2] if num_task == 2 else [y1])

    t = df_T.loc[df_T['fips']==id][score].values[0]
    # t = 1 if t > 4 else 0 # binarize T (A, B --> 1, C, D, F --> 0)

    id_key = "{:05}".format(id)
    data[id_key] = (relevant_x, y, t)
  
  return data


In [0]:
df_X, df_Y, df_T = load_data(ver=2, t_type='unacast')
data = construct_dataset(df_X, df_Y, df_T, Y_mode='adj_increase', num_task=3, t_type='unacast')

print(len(data))
print(data['25025'][0]) # Suffolk County, Massachusetts
print(data['25025'][1]) # Suffolk County, Massachusetts
print(data['25025'][2]) # Suffolk County, Massachusetts

2524
[0.24667761739828456, 0.23381050774727105, 0.44799999999999995, 0.1649348654447434, 0.03, 13859.478078, 0.05627073538980396, 0.2002966098421943, 0.0020117633650954102, 1.8581558175142334e-05, 0.0015947118261191904]
[0.47606786072802665, 0.4851307427351671, 0.5294974004041098]
8.0


In [0]:
# Split Train, Val, Test
import random
# test = {'0': 0, '1': 1, '2':2, '3':3}
# keys = list(test.keys())
# shuffledKeys = random.sample(keys, 4)
# print(shuffledKeys)

def generate_split(keys, val_size=0.1, test_size=0.2):
  if val_size + test_size >= 1.0:
    raise ValueError("Sum of the validation size and the test size should be less than 1")
  shuffled_keys = random.sample(keys, len(keys))

  total_len = len(keys)
  train_keys = shuffled_keys[:int(total_len*(1-val_size-test_size))]
  val_keys = shuffled_keys[int(total_len*(1-val_size-test_size)):int(total_len*(1-test_size))]
  test_keys = shuffled_keys[int(total_len*(1-test_size)):]

  return train_keys, val_keys, test_keys

train_keys, val_keys, test_keys = generate_split(list(data.keys()))
print(len(train_keys), len(val_keys), len(test_keys))

1766 253 505


In [0]:
def construct_split_dataset(data, train_keys, val_keys, test_keys, censor_by='all'):
  XT_train = []
  XT_val = []
  XT_test = []

  Y_train = []
  Y_val= []
  Y_test = []

  final_train_keys = []
  for key in train_keys:
    x, y, t = data[key]
    y = np.asarray(y)
    last_three_x = x[len(x)-3:]
    flag = (last_three_x[0]> 0 and last_three_x[1]>0 and last_three_x[2]>0) if censor_by == 'all' else (last_three_x[0]> 0 and last_three_x[2]>0)
    if not flag:
      continue
      
    feature = np.insert(x, len(x), t)
    if np.isnan(feature.sum()):
      continue
    if np.isnan(y.sum()):
      continue
    if not (y[0] > 0 and y[1] > 0 and y[2] > 0 ):
      continue
    XT_train.append(feature)
    Y_train.append(y)
    final_train_keys.append(key)

  final_val_keys = []
  for key in val_keys:
    x, y, t = data[key]
    y = np.asarray(y)
    last_three_x = x[len(x)-3:]
    flag = (last_three_x[0]> 0 and last_three_x[1]>0 and last_three_x[2]>0) if censor_by == 'all' else (last_three_x[0]> 0 and last_three_x[2]>0)
    if not flag:
      continue

    feature = np.insert(x, len(x), t)
    if np.isnan(feature.sum()):
      continue
    if np.isnan(y.sum()):
      continue
    if not (y[0] > 0 and y[1] > 0 and y[2] > 0):
      continue
    XT_val.append(feature)
    Y_val.append(y)
    final_val_keys.append(key)

  final_test_keys = []
  for key in test_keys:
    x, y, t = data[key]
    y = np.asarray(y)
    last_three_x = x[len(x)-3:]
    flag = (last_three_x[0]> 0 and last_three_x[1]>0 and last_three_x[2]>0) if censor_by == 'all' else (last_three_x[0]> 0 and last_three_x[2]>0)
    if not flag:
      continue

    feature = np.insert(x, len(x), t)
    if np.isnan(feature.sum()):
      continue
    if np.isnan(y.sum()):
      continue
    if not (y[0] > 0 and y[1] > 0 and y[2] > 0):
      continue
    XT_test.append(feature)
    Y_test.append(y)
    final_test_keys.append(key)

  XT_train = np.asarray(XT_train)
  XT_val = np.asarray(XT_val)
  XT_test = np.asarray(XT_test)
  Y_train = np.asarray(Y_train)
  Y_val = np.asarray(Y_val)
  Y_test = np.asarray(Y_test)
  return XT_train, XT_val, XT_test, Y_train, Y_val, Y_test, final_train_keys, final_val_keys, final_test_keys

In [0]:
XT_train, XT_val, XT_test, Y_train, Y_val, Y_test, final_train_keys, final_val_keys, final_test_keys = \
construct_split_dataset(data, train_keys, val_keys, test_keys, censor_by='death-only')
print(len(XT_train))
print(XT_train[0])
print(len(XT_val), len(XT_test))
print(len(final_train_keys))

911
[2.80213615e-01 1.48597745e-01 2.61000000e-01 1.54820516e-01
 4.40000000e-02 1.46808239e+03 4.63246822e-02 1.46383042e-01
 3.30145536e-03 1.47448111e-04 2.48327719e-03 7.00000000e+00]
128 251
911


In [0]:
# Train the MultiTaskLasso
from sklearn import linear_model, metrics, multioutput, neighbors

def train_model(XT_train, Y_train, XT_val, Y_val, scaler, y_scaler, alpha, l1_ratio, y_scale=False, model_type='linear', n_neighbors=20):
  if model_type == 'linear':
    # model = linear_model.MultiTaskLasso(alpha=alpha, max_iter=100000)
    model = linear_model.MultiTaskElasticNet(alpha=alpha, l1_ratio=l1_ratio)
    # model = multioutput.RegressorChain(linear_model.Ridge(alpha=alpha, max_iter=100000))
  elif model_type == 'knn':
    basemodel = neighbors.KNeighborsRegressor(n_neighbors=n_neighbors, weights='distance')
    model = multioutput.RegressorChain(basemodel)
  XT_train_transformed = scaler.transform(XT_train)
  # print(XT_train_transformed.mean(axis=0))
  if y_scale:
    Y_train_ = y_scaler.transform(Y_train)
  else:
    Y_train_ = Y_train

  model.fit(XT_train_transformed, Y_train_)

  pred = model.predict(XT_train_transformed)
  mse = metrics.mean_squared_error(Y_train_, pred)
  print("Mean Squred Error for the Train Set: {:.4f}".format(mse))
  print("R^2 for this model (closer to 1, the better): {:.4f}".format(model.score(XT_train_transformed, Y_train_)))

  XT_val_transformed = scaler.transform(XT_val)
  if y_scale:
    Y_val_ = y_scaler.transform(Y_val)
  else:
    Y_val_ = Y_val
  pred = model.predict(XT_val_transformed)
  mse = metrics.mean_squared_error(Y_val_, pred)
  print("Mean Squred Error for the Validation Set: {:.4f}".format(mse))
  print("R^2 for this model (closer to 1, the better): {:.4f}".format(model.score(XT_val_transformed, Y_val_)))
  # print("Printing predictions on the validataion set")
  # print("Index , [Ground Truth], [Predicted]")
  # Y_val_ = y_scaler.inverse_transform(Y_val_)
  # pred = y_scaler.inverse_transform(pred)
  # for idx, (y, p) in enumerate(zip(Y_val_, pred)):
  #   print(idx, y, p)

  return model

def test_model(model, XT, Y, scaler, y_scaler, y_scale=True):
  XT_transformed = scaler.transform(XT)
  if y_scale:
    Y_ = y_scaler.transform(Y)
  else:
    Y_ = y
  pred = model.predict(XT_transformed)
  mse = metrics.mean_squared_error(Y_, pred)
  print("Mean Squred Error for the Test Set: {:.4f}".format(mse))
  print("R^2 for this model (closer to 1, the better): {:.4f}".format(model.score(XT_transformed, Y_)))
  print("Printing predictions on the validataion set")
  print("Index , [Ground Truth], [Predicted]")
  Y_val_ = y_scaler.inverse_transform(Y_)
  pred = y_scaler.inverse_transform(pred)
  for idx, (y, p) in enumerate(zip(Y_val_, pred)):
    print(idx, y, p)


Including Networking Effect with Simple Markov Assumption

In [0]:
import csv
PATH = '/content/drive/My Drive/COVID19 Data Exploration /Data/county_adjacency.csv'
county_df = pd.read_csv(PATH, header=None)
county_df.head()

,0,1
0,1001.0,1001
1,NaN,1021
2,NaN,1047
3,NaN,1051
4,NaN,1085


In [0]:
county_adjacency = {}
prev_fips = float('nan')
for idx, row in county_df.iterrows():
  fips = row[0]
  neighbor = row[1]
  if math.isnan(fips):
    fips = prev_fips
  else:
    prev_fips = fips
  if int(fips) in county_adjacency.keys():
    county_adjacency[int(fips)].append(int(neighbor))
  else:
    county_adjacency[int(fips)] = [int(neighbor)]

print(county_adjacency[1001])


[1001, 1021, 1047, 1051, 1085, 1101]


In [0]:
def check_data_validity(x, y, t, censor_by):
  last_three_x = x[len(x)-3:]
  flag = (last_three_x[0]> 0 and last_three_x[1]>0 and last_three_x[2]>0) if censor_by == 'all' else (last_three_x[0]> 0 and last_three_x[2]>0)
  if not flag:
    return False
  feature = np.insert(x, len(x), t)
  if np.isnan(feature.sum()):
    return False
  if np.isnan(y.sum()):
    return False
  if not (y[0] > 0 and y[1] > 0 and y[2] > 0):
    return False

  return True

def process_data_with_adjacency(data, keys, adjacency, censor_by='all', neighbor_strength=1.0):
  XT = []
  Y = []
  final_keys = []

  for key in keys:
    x, y, t = data[key]
    y = np.asarray(y)
    valid = check_data_validity(x, y, t, censor_by)
    if valid == False:
      continue
    xt = np.insert(x, len(x), t)
    final_keys.append(key)

    adj_xt= []
    if int(key) in adjacency.keys():
      neighbors = adjacency[int(key)]
      for n in neighbors:
        # Exclude itself
        if int(n) == int(key):
          continue
        if "{:05}".format(n) in data.keys():
          a_x, a_y, a_t = data["{:05}".format(n)]
          a_y = np.asarray(a_y)
          a_valid = check_data_validity(a_x, a_y, a_t, censor_by)
          if a_valid:
            a_xt = np.insert(a_x, len(a_x), a_t)
            adj_xt.append(a_xt)
      
    if len(adj_xt) > 0:
      # print(len(adj_xt))
      adj_xt = np.asarray(adj_xt)
      adj_effect = adj_xt.mean(axis=0)
      xt = xt + neighbor_strength * adj_effect
    else:
      xt = (1 + neighbor_strength) * xt
    
    XT.append(xt)
    Y.append(y)

  return XT, Y, final_keys


In [0]:
neighbor_XT_train, neighbor_Y_train, neighbor_final_keys = process_data_with_adjacency(data, train_keys, county_adjacency, \
                                                                                       censor_by='death-only', neighbor_strength=1.0)


In [0]:
neighbor_XT_val, neighbor_Y_val, neighbor_final_val = process_data_with_adjacency(data, val_keys, county_adjacency, censor_by='death-only', neighbor_strength=1.0)
neighbor_XT_test, neighbor_Y_test, neighbor_final_test = process_data_with_adjacency(data, test_keys, county_adjacency, censor_by='death-only', neighbor_strength=1.0)

In [0]:
# Standaradization for XT
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(neighbor_XT_train)
y_scaler = preprocessing.StandardScaler().fit(neighbor_Y_train)

print(scaler.mean_)
print(y_scaler.mean_)

for alpha in [0, 1e-3, 2e-3, 5e-3, 1e-2, 2e-2, 5e-2]:
  for l1 in [0, 0.01, 0.1, 0.2, 0.5]:
    print(alpha, l1)
    model = train_model(neighbor_XT_train, neighbor_Y_train, neighbor_XT_val, neighbor_Y_val, scaler, y_scaler, alpha, l1, True)
# coef = model.coef_
# for i in range(3):
#   c = coef[i]
#   print(c.shape)
#   print("------------------------------------")
#   print("{}th Task".format(i))
#   # print("Sex (Male ratio): {:.5f}".format(c[0]))
#   # print("Age (<60 ratio) : {:.5f}".format(c[1]))
#   # print("Race - White    : {:.5f}".format(c[2]))
#   print("Race - Black    : {:.5f}".format(c[0]))
#   # print("Race - Native   : {:.5f}".format(c[4]))
#   # print("Race - Pacific  : {:.5f}".format(c[5]))
#   # print("Race - Asian    : {:.5f}".format(c[6]))
#   print("Race - Hispanic : {:.5f}".format(c[1]))
#   print("Bachelor degree : {:.5f}".format(c[2]))
#   print("Poverty         : {:.5f}".format(c[3]))
#   print("Unemployment    : {:.5f}".format(c[4]))
#   # print("Median Income   : {:.5f}".format(c[11]))
#   print("Pop density     : {:.5f}".format(c[5]))
#   print("Test / capita   : {:.5f}".format(c[6]))
#   print("Postive test    : {:.5f}".format(c[7]))
#   # print("Politic - Dem   : {:.5f}".format(c[15]))
#   print("Covid - Case    : {:.5f}".format(c[8]))
#   print("Covid - Death   : {:.5f}".format(c[9]))
#   print("Covid - Increase: {:.5f}".format(c[10]))
#   print("TREATMENT       : {:.5f}".format(c[11]))

[2.68996362e-01 2.09688359e-01 5.24702732e-01 2.72803607e-01
 8.16409789e-02 8.54013682e+02 5.43073215e-02 2.50117104e-01
 7.98574043e-04 2.08007457e-05 5.60725478e-04 1.01580881e+01]
[0.06314089 0.06943262 0.06961384]
0 0


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:1790: UserWarning: Coordinate descent with l1_reg=0 may lead to unexpected results and is discouraged.
  check_random_state(self.random_state), random)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:1790: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 841.7411814616121, tolerance: 0.27330000000000026
  check_random_state(self.random_state), random)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_aver

Mean Squred Error for the Train Set: 0.6160
R^2 for this model (closer to 1, the better): 0.3840
Mean Squred Error for the Validation Set: 0.2591
R^2 for this model (closer to 1, the better): 0.4965
0 0.01


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:1790: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 841.7411814616121, tolerance: 0.27330000000000026
  check_random_state(self.random_state), random)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' t

Mean Squred Error for the Train Set: 0.6160
R^2 for this model (closer to 1, the better): 0.3840
Mean Squred Error for the Validation Set: 0.2591
R^2 for this model (closer to 1, the better): 0.4965
0 0.1


...

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multi

Mean Squred Error for the Train Set: 0.6176
R^2 for this model (closer to 1, the better): 0.3824
Mean Squred Error for the Validation Set: 0.2604
R^2 for this model (closer to 1, the better): 0.4940
0.01 0.01
Mean Squred Error for the Train Set: 0.6176
R^2 for this model (closer to 1, the better): 0.3824
Mean Squred Error for the Validation Set: 0.2604
R^2 for this model (closer to 1, the better): 0.4941
0.01 0.1
Mean Squred Error for the Train Set: 0.6177
R^2 for this model (closer to 1, the better): 0.3823
Mean Squred Error for the Validation Set: 0.2600
R^2 for this model (closer to 1, the better): 0.4948
0.01 0.2
Mean Squred Error for the Train Set: 0.6178
R^2 for this model (closer to 1, the better): 0.3822
Mean Squred Error for the Validation Set: 0.2596
R^2 for this model (closer to 1, the better): 0.4955
0.01 0.5
Mean Squred Error for the Train Set: 0.6182
R^2 for this model (closer to 1, the better): 0.3818
Mean Squred Error for the Validation Set: 0.2586
R^2 for this model (c

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:1790: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 853.182019183806, tolerance: 0.27330000000000026
  check_random_state(self.random_state), random)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to

Mean Squred Error for the Train Set: 0.6190
R^2 for this model (closer to 1, the better): 0.3810
Mean Squred Error for the Validation Set: 0.2613
R^2 for this model (closer to 1, the better): 0.4923
0.02 0.01
Mean Squred Error for the Train Set: 0.6190
R^2 for this model (closer to 1, the better): 0.3810
Mean Squred Error for the Validation Set: 0.2612
R^2 for this model (closer to 1, the better): 0.4925
0.02 0.1
Mean Squred Error for the Train Set: 0.6190
R^2 for this model (closer to 1, the better): 0.3810
Mean Squred Error for the Validation Set: 0.2604
R^2 for this model (closer to 1, the better): 0.4939
0.02 0.2
Mean Squred Error for the Train Set: 0.6191
R^2 for this model (closer to 1, the better): 0.3809
Mean Squred Error for the Validation Set: 0.2596
R^2 for this model (closer to 1, the better): 0.4955
0.02 0.5
Mean Squred Error for the Train Set: 0.6193
R^2 for this model (closer to 1, the better): 0.3807
Mean Squred Error for the Validation Set: 0.2572
R^2 for this model (c

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_coordinate_descent.py:1790: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 862.2633425798007, tolerance: 0.27330000000000026
  check_random_state(self.random_state), random)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' t

In [0]:
model = train_model(neighbor_XT_train, neighbor_Y_train, neighbor_XT_val, neighbor_Y_val, scaler, y_scaler, 0.05, 0.5, True)
test_model(model, neighbor_XT_test, neighbor_Y_test, scaler, y_scaler, y_scale=True)

Mean Squred Error for the Train Set: 0.6233
R^2 for this model (closer to 1, the better): 0.3767
Mean Squred Error for the Validation Set: 0.2557
R^2 for this model (closer to 1, the better): 0.5032
Mean Squred Error for the Test Set: 0.4869
R^2 for this model (closer to 1, the better): -0.0715
Printing predictions on the validataion set
Index , [Ground Truth], [Predicted]
0 [0.07860523 0.14148941 0.20699377] [0.10517137 0.11366516 0.11219537]
1 [0.27460873 0.26703332 0.33584332] [0.20174315 0.18176944 0.17567811]
2 [0.02396472 0.01711766 0.03651767] [0.0708539  0.07467847 0.082526  ]
3 [0.01417062 0.0139738  0.0139082 ] [0.01959886 0.03236169 0.03394471]
4 [0.03912899 0.03353913 0.03540242] [0.05467811 0.06541469 0.06194942]
5 [0.05461166 0.0451316  0.04380817] [0.20326557 0.25256456 0.25383235]
6 [0.01683231 0.01424273 0.0202851 ] [0.0250577  0.03820236 0.03869173]
7 [0.02497721 0.01561076 0.0124886 ] [0.01550475 0.02940297 0.03003148]
8 [0.04413645 0.04271923 0.04805069] [0.06113373

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multioutput='uniform_average').", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/base.py:434: FutureWarning: The default value of multioutput (not exposed in score method) will change from 'variance_weighted' to 'uniform_average' in 0.23 to keep consistent with 'metrics.r2_score'. To specify the default value manually and avoid the warning, please either call 'metrics.r2_score' directly or make a custom scorer with 'metrics.make_scorer' (the built-in scorer 'r2' uses multioutput='uniform_average').
  "multi

In [0]:
print(model.coef_)

[[ 9.36225136e-02  0.00000000e+00 -5.25975035e-02  4.09787140e-02
   1.61334554e-03  1.41290324e-01 -5.41427326e-05  1.15842536e-01
   6.74841460e-02  9.64189265e-03  5.76936965e-01 -3.81026583e-03]
 [ 7.33171508e-02  0.00000000e+00 -6.04269938e-02  1.73320301e-02
   1.01025297e-03  1.18534960e-01 -1.37844495e-02  8.51507021e-02
   4.83352972e-02 -2.02308346e-02  2.99874504e-01 -9.08533996e-03]
 [ 8.41822884e-02  0.00000000e+00 -5.36796805e-02  2.15980575e-02
   8.52404009e-04  1.28500756e-01 -7.41199630e-03  1.06504131e-01
   5.84230802e-02 -3.23487458e-02  2.84261069e-01 -1.15311534e-02]]


In [0]:
def cate_measurement(model, scaler, y_scaler, fips, data, df_X, adjacency, neighbor_t=False, censor_by='death-only', neighbor_strength=1.0):
  # Return the expected Y's when T is different (1~9 for Unacast)
  STNAME = df_X.loc[df_X['fips']==int(fips)]['STNAME'].values[0]
  CTYNAME = df_X.loc[df_X['fips']==int(fips)]['CTYNAME'].values[0]

  print("Fips [{}] {} - {}".format(fips, STNAME, CTYNAME))
  x, y, t = data[fips]
  y = np.asarray(y)
  print("Original T: {}".format(t))
  print("Original Y1, Y2, Y3: {}, {}, {}".format(y[0], y[1], y[2]))

  pred_1 = []
  pred_9 = []
  for tt in range(1, 10):
    feature = np.insert(x, len(x), tt)
    adj_xt= []
    if int(fips) in adjacency.keys():
      neighbors = adjacency[int(fips)]
      for n in neighbors:
        # Exclude itself
        if int(n) == int(fips):
          continue
        if "{:05}".format(n) in data.keys():
          a_x, a_y, a_t = data["{:05}".format(n)]
          if neighbor_t:
            a_t = tt
          a_y = np.asarray(a_y)
          a_valid = check_data_validity(a_x, a_y, a_t, censor_by)
          if a_valid:
            a_xt = np.insert(a_x, len(a_x), a_t)
            adj_xt.append(a_xt)
      
    if len(adj_xt) > 0:
      # print(len(adj_xt))
      adj_xt = np.asarray(adj_xt)
      adj_effect = adj_xt.mean(axis=0)
      feature = feature + neighbor_strength * adj_effect
    else:
      feature = (1+neighbor_strength) * feature

    XT = scaler.transform(feature.reshape((1, -1)))
    pred = model.predict(XT)
    pred = y_scaler.inverse_transform(pred)[0]
    print("When T={}, Y's are: {}, {}, {}".format(tt, pred[0], pred[1], pred[2]))

    if tt == 1:
      pred_1 = pred
    if tt == 9:
      pred_9 = pred

  return pred_1, pred_9

In [0]:
cate_1 = 0
cate_2 = 0
cate_3 = 0
for key in neighbor_final_test:
  pred_1, pred_9 = cate_measurement(model, scaler, y_scaler, key, data, df_X, county_adjacency, neighbor_t=True)
  print("----------------------------------------------------------")
  cate_1 += pred_9[0] - pred_1[0]
  cate_2 += pred_9[1] - pred_1[1]
  cate_3 += pred_9[2] - pred_1[2]

cate_1 /= float(len(neighbor_final_test))
cate_2 /= float(len(neighbor_final_test))
cate_3 /= float(len(neighbor_final_test))

print(cate_1*100, cate_2*100, cate_3*100)

Fips [28007] Mississippi - Attala County
Original T: 5.0
Original Y1, Y2, Y3: 0.0786052288198211, 0.14148941187567798, 0.2069937692255289
When T=1, Y's are: 0.1062442295328975, 0.11769634001635011, 0.11685931862109344
When T=2, Y's are: 0.10596863274691322, 0.11666080761844579, 0.11566124006744424
When T=3, Y's are: 0.10569303596092894, 0.11562527522054147, 0.11446316151379504
When T=4, Y's are: 0.10541743917494464, 0.11458974282263715, 0.11326508296014581
When T=5, Y's are: 0.10514184238896036, 0.11355421042473282, 0.11206700440649661
When T=6, Y's are: 0.10486624560297608, 0.11251867802682851, 0.11086892585284741
When T=7, Y's are: 0.10459064881699179, 0.1114831456289242, 0.10967084729919818
When T=8, Y's are: 0.1043150520310075, 0.11044761323101987, 0.10847276874554898
When T=9, Y's are: 0.10403945524502323, 0.10941208083311554, 0.10727469019189978
----------------------------------------------------------
Fips [36105] New York - Sullivan County
Original T: 3.0
Original Y1, Y2, Y3: 

In [0]:
cate_measurement(model, scaler, y_scaler, '25025', data, df_X, county_adjacency, neighbor_t=True)
cate_measurement(model, scaler, y_scaler, '25025', data, df_X, county_adjacency, neighbor_t=False)

Fips [25025] Massachusetts - Suffolk County
Original T: 8.0
Original Y1, Y2, Y3: 0.47606786072802665, 0.4851307427351671, 0.5294974004041098
When T=1, Y's are: 0.3438335122631082, 0.3607266442513403, 0.3554105269734966
When T=2, Y's are: 0.34355791547712394, 0.35969111185343605, 0.35421244841984745
When T=3, Y's are: 0.3432823186911397, 0.35865557945553167, 0.3530143698661983
When T=4, Y's are: 0.34300672190515535, 0.3576200470576274, 0.351816291312549
When T=5, Y's are: 0.34273112511917103, 0.3565845146597231, 0.3506182127588998
When T=6, Y's are: 0.3424555283331868, 0.35554898226181875, 0.34942013420525064
When T=7, Y's are: 0.34217993154720255, 0.3545134498639144, 0.34822205565160136
When T=8, Y's are: 0.3419043347612183, 0.3534779174660101, 0.3470239770979522
When T=9, Y's are: 0.34162873797523396, 0.3524423850681058, 0.345825898544303
Fips [25025] Massachusetts - Suffolk County
Original T: 8.0
Original Y1, Y2, Y3: 0.47606786072802665, 0.4851307427351671, 0.5294974004041098
When T=

(array([0.34295505, 0.35742588, 0.35159165]),
 array([0.34185266, 0.35328376, 0.34679934]))